In [1]:
from prometheus_api_client import PrometheusConnect
from datetime import datetime
import pandas as pd
import os

In [ ]:
#prom = PrometheusConnect(url ="http://prometheus-openshift.viavarejo.com.br", disable_ssl=True)
#prom = PrometheusConnect(url ="http://prometheus-akspriv-checkout-prd.viavarejo.com.br", disable_ssl=True)

In [2]:
# função para identificar pico de requests dentro do range de tempo passado e gerar a média de reqs/s a partir de 10 min do pico

def mean_requests_max_nginx(app, data_inicio, data_fim):
    prom = PrometheusConnect(url =f"http://'prometheus-akspriv-checkout-prd'.viavarejo.com.br", disable_ssl=True)
    # criando dataframe com a quantidade de reqs/s, data e posição na tabela
    request_count_nginx = pd.DataFrame()
    for posicao, numero in enumerate(list(prom.custom_query_range(query='sum(rate(nginx_ingress_controller_requests{service="' + app + '", status="200"}[1m]))', start_time=datetime.strptime(data_inicio, '%Y/%m/%d %H:%M'), end_time=datetime.strptime(data_fim, '%Y/%m/%d %H:%M'), step='1m'))[0]['values']):
        request_count_nginx = request_count_nginx.append({
            'posicao': posicao,
            'data': numero[0],
            'valor': numero[1]
        }, ignore_index=True)
        

    # identificando pico de requests/s dentro do período de tempo passado e tirando uma média de reqs/s dos últimos 10 min a partir do pico 
    valores_pico = pd.DataFrame()
    for n in range(len(request_count_nginx)):
        if (request_count_nginx['posicao'].iloc[n] >= int(request_count_nginx[request_count_nginx['valor'] == max(request_count_nginx['valor'])]['posicao'].iloc[0]) - 10) & (request_count_nginx['posicao'].iloc[n] <= int(request_count_nginx[request_count_nginx['valor'] == max(request_count_nginx['valor'])]['posicao'].iloc[0])):
            valores_pico = valores_pico.append({
                'valor': float(request_count_nginx['valor'].iloc[n])
            }, ignore_index=True) 
    
    #return request_count_nginx[request_count_nginx['valor'] == max(request_count_nginx['valor'])]['posicao'].iloc[0], request_count_nginx[request_count_nginx['valor'] == max(request_count_nginx['valor'])]['data'].iloc[0] ,max(request_count_nginx['valor'])
    return round(valores_pico['valor'].mean() * 60) 

In [3]:
compra = [
    {'jornada': 'Compra', 'tribo': 'vitrine', 'produto': 'carrinho', 'reqs_cb': 'carrinho-api-cb', 'reqs_ex': 'carrinho-api-ex', 'reqs_pf': 'carrinho-api-pf', 'reqs_corp': ''},

    {'jornada': 'Compra', 'tribo': 'carrinho-front', 'produto': 'apollo', 'reqs_cb': 'carrinho-front-cb', 'reqs_ex': 'carrinho-front-ex', 'reqs_pf': 'carrinho-front-pf', 'reqs_corp': ''},

    {'jornada': 'Compra', 'tribo': 'check', 'produto': 'checkout', 'reqs_cb': 'checkout-api-cb', 'reqs_ex': 'checkout-api-ex', 'reqs_pf': 'checkout-api-pf', 'reqs_corp': ''},
]

b2b = [
    {'jornada': 'b2b', 'tribo': 'parcerias', 'produto': 'checkout-front', 'reqs_cb': 'checkout-front-cb', 'reqs_ex': 'checkout-front-ex', 'reqs_pf': 'checkout-front-pf', 'reqs_corp': ''},

    {'jornada': 'b2b', 'tribo': 'parcerias', 'produto': 'televendas', 'reqs_cb': '', 'reqs_ex': '', 'reqs_pf': '', 'reqs_corp': 'televendas-api-sit'},

    {'jornada': 'b2b', 'tribo': 'parcerias', 'produto': 'kong', 'reqs_cb': '', 'reqs_ex': '', 'reqs_pf': '', 'reqs_corp': 'kong-data-plane-proxy'},
]

figital = [
    {'jornada': 'teste', 'tribo': 'teste', 'produto': 'teste', 'reqs_cb': 'checkout-front-cb', 'reqs_ex': 'checkout-front-ex', 'reqs_pf': 'checkout-front-pf', 'reqs_corp': ''},

    {'jornada': 'teste', 'tribo': 'teste', 'produto': 'teste', 'reqs_cb': '', 'reqs_ex': '', 'reqs_pf': '', 'reqs_corp': 'televendas-api-sit'},

    {'jornada': 'teste', 'tribo': 'teste', 'produto': 'teste', 'reqs_cb': '', 'reqs_ex': '', 'reqs_pf': '', 'reqs_corp': 'kong-data-plane-proxy'},
]

In [4]:
df = pd.concat([pd.DataFrame(data=compra), pd.DataFrame(data=b2b), pd.DataFrame(data=figital)])

In [5]:
df

,jornada,tribo,produto,reqs_cb,reqs_ex,reqs_pf,reqs_corp
0,Compra,vitrine,carrinho,carrinho-api-cb,carrinho-api-ex,carrinho-api-pf,
1,Compra,carrinho-front,apollo,carrinho-front-cb,carrinho-front-ex,carrinho-front-pf,
2,Compra,check,checkout,checkout-api-cb,checkout-api-ex,checkout-api-pf,
0,b2b,parcerias,checkout-front,checkout-front-cb,checkout-front-ex,checkout-front-pf,
1,b2b,parcerias,televendas,,,,televendas-api-sit
2,b2b,parcerias,kong,,,,kong-data-plane-proxy
0,teste,teste,teste,checkout-front-cb,checkout-front-ex,checkout-front-pf,
1,teste,teste,teste,,,,televendas-api-sit
2,teste,teste,teste,,,,kong-data-plane-proxy


In [6]:
data_hora_inicio = '2023/03/24 12:00'
data_hora_fim = '2023/03/24 13:00'
#server = 'prometheus-akspriv-checkout-prd'

for numero, coluna in enumerate(df):
    if numero > 2:
        for num, linha in enumerate(df[coluna]):
            try:
                df[coluna].iloc[num] = mean_requests_max_nginx(linha, data_hora_inicio, data_hora_fim)
                #print(linha, mean_requests_max_nginx(linha, data_hora_inicio, data_hora_fim))
            except:
                #print(linha, 'excepção')
                df[coluna].iloc[num] = ''
                #pass

In [7]:
df

,jornada,tribo,produto,reqs_cb,reqs_ex,reqs_pf,reqs_corp
0,Compra,vitrine,carrinho,,,,
1,Compra,carrinho-front,apollo,,,,
2,Compra,check,checkout,,,,
0,b2b,parcerias,checkout-front,,,,
1,b2b,parcerias,televendas,,,,
2,b2b,parcerias,kong,,,,
0,teste,teste,teste,,,,
1,teste,teste,teste,,,,
2,teste,teste,teste,,,,
